In [ ]:
## Optimization: Selected multi-step classification set
##############################
######### YM dataset #########
##############################

# Boruta-SHAP
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
from sklearn import preprocessing
from sklearn.metrics import roc_curve, roc_auc_score, auc, recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

import csv

from matplotlib import pyplot

from BorutaShap import BorutaShap

# Creates a BorutaShap selector for regression
selector = BorutaShap(model = None, importance_measure = 'shap', classification = True)


for k in range(0,2):
    print(k)

    if k == 0:
        ym = pd.read_csv('dt_ym_12_3_4_5_new.csv',';')
    elif k == 1:
        ym = pd.read_csv('dt_ym_123_4_5_new.csv',';')  
        
    
#   # Accuracy 70% achieved with this multi-steps
#     if k == 0:
#         ym = pd.read_csv('dt_ym_123_45_new.csv',';')
#     elif k == 1:
#         ym = pd.read_csv('dt_ym_4_5_new.csv',';')
#     elif k == 2:
#         ym = pd.read_csv('dt_ym_12_3_new.csv',';')
#     elif k == 3:
#         ym = pd.read_csv('dt_ym_1_2_new.csv',';')

#   # Accuracy 72% achieved with this multi-steps
#     if k == 0:
#         ym = pd.read_csv('dt_ym_1245_3_new.csv',';')
#     elif k == 1:
#         ym = pd.read_csv('dt_ym_124_5_new.csv',';')
#     elif k == 2:
#         ym = pd.read_csv('dt_ym_12_4_new.csv',';')
#     elif k == 3:
#         ym = pd.read_csv('dt_ym_1_2_new.csv',';')

        
        
    ym.head()

    ym_pr = ym
    X = ym_pr.drop(columns = ['pref_nominal', 'ac2_nominal', 'ac5_nominal', 'preference', 'action2', 'action5', 'action5_1'])
    ym_ac5_1_cl = ym['action5_1']
    
    y = ym_ac5_1_cl

    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)
    
    
############################# basic model training and test result to plot model loss ################################
    model_base = XGBClassifier(random_state=42)
    evalset = [(X_train, y_train), (X_test, y_test)]
    model_base.fit(X_train, y_train, eval_metric='mlogloss', eval_set=evalset)
    results_base = model_base.evals_result()
#######################################################################################################################    
    
        

    # y_train2 = y_train.map({'prefer': 1, 'not': 0}).astype(int) 
    # y_pred2 = pd.Series(y_pred2)
    # y_pred2 = y_pred2.map({'prefer': 1, 'not': 0}).astype(int) 
    # y_test = y_test.map({'prefer': 1, 'not': 0}).astype(int)


    # Fits the selector
    selector.fit(X = X_train, y = y_train, n_trials = 100, sample = False, verbose = True)
    # n_trials -> number of iterations for Boruta algorithm
    # sample -> samples the data so it goes faster

    # Display features to be removed
    features_to_remove = selector.features_to_remove
    print(features_to_remove)
    # Removes them
    X_train_boruta_shap = X_train.drop(columns = features_to_remove)
    X_test_boruta_shap = X_test.drop(columns = features_to_remove)


      
    model = XGBClassifier(random_state=42)
    evalset = [(X_train_boruta_shap, y_train), (X_test_boruta_shap, y_test)]
    # fit the model
    model.fit(X_train_boruta_shap, y_train, eval_metric='mlogloss', eval_set=evalset, early_stopping_rounds=30)


    #     # y_test = y_test.map({'prefer': 1, 'not': 0}).astype(int)
    #     fpr, tpr, _ = roc_curve(y_test.values, pred_p, pos_label=1)
    #     roc_auc = auc(fpr, tpr)


  
    ################################ Bayesian optimization: Hyperparameter ######################################    
  
    # ym_pr_cl  = ym['pref_nominal']
    # ym_pr_cl  = ym['preference']
    
    y = ym_ac5_1_cl
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)
    
    
    pbounds = {
    'learning_rate': (0.01, 0.4),
    'n_estimators': (20, 100),
    'max_depth': (1,5),
    #     'subsample': (1.0, 1.0),  # Change for big datasets
    #     'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 1)}


    def xgboost_hyper_param(learning_rate,
                            n_estimators,
                            max_depth,
    #                         subsample,
    #                         colsample,
                            gamma):

        max_depth = int(max_depth)
        n_estimators = int(n_estimators)

        clf = XGBClassifier(
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            gamma=gamma)
        return np.mean(cross_val_score(clf, X_train_boruta_shap, y_train, cv=3, scoring='roc_auc_ovr'))

    optimizer = BayesianOptimization(
        f=xgboost_hyper_param,
        pbounds=pbounds,
        random_state=1)


    # performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
    ## optimizer.maximize(n_iter=10, init_points=20, acq='ei')
    optimizer.set_gp_params(alpha=1e-3, n_restarts_optimizer=5)
    optimizer.maximize(init_points=2, n_iter=10)

    #Extracting the best parameters
    params = optimizer.max['params']
    print(params)

    #Converting the max_depth and n_estimator values from float to int
    params['max_depth']= int(params['max_depth'])
    params['n_estimators']= int(params['n_estimators'])
#     learning_rate = 0.05
#     params['learning_rate']= int(learning_rate)
    

    #Initialize an XGBClassifier with the tuned parameters and fit the training data
    # from xgboost import XGBClassifier
    evalset = [(X_train_boruta_shap, y_train), (X_test_boruta_shap, y_test)]
    model2 = XGBClassifier(**params).fit(X_train_boruta_shap, y_train, eval_metric='mlogloss', eval_set=evalset, early_stopping_rounds=30)

    #predicting for training set
    y_pred2 = model2.predict(X_train_boruta_shap)

    #Looking at the classification report
    print(classification_report(y_pred2, y_train))
    
        
    # pandas.Series
    score = model2.score(X_test_boruta_shap, y_test)

    # make predictions for test data
    y_pred2 = model2.predict(X_test_boruta_shap)
    pred_p2 = model2.predict_proba(X_test_boruta_shap)

    y_pred2 = pd.Series(y_pred2)
#     y_pred2 = y_pred2.map({'prefer': 1, 'not': 0}).astype(int) 
#     y_test = y_test.map({'prefer': 1, 'not': 0}).astype(int)
    valid_score = roc_auc_score(y_test, pred_p2, multi_class="ovr")

    #Looking at the classification report
    print(classification_report(y_pred2, y_test))
    
    
#     print('Precision: %.3f' % precision_score(y_test, y_pred2))
#     print('Recall: %.3f' % recall_score(y_test, y_pred2))
#     print('F1 Score: %.3f' % f1_score(y_test, y_pred2))
#     print('Accuracy: %.3f' % accuracy_score(y_test, y_pred2))
#     print('Validation ROC-AUC score: %.3f' % valid_score)
     
        
    # retrieve performance metrics
    results = model2.evals_result()

    if k == 0:    
        young_results = model2.evals_result()
        young_params = params

    elif k == 1:
        old_results = model2.evals_result()
        old_params = params
    
    elif k ==2:
        comb_results = model2.evals_result()
        comb_params = params    
    
    
    # plot learning curves
    pyplot.plot(results['validation_0']['mlogloss'], label='train')
    pyplot.plot(results['validation_1']['mlogloss'], label='test')
    # show the legend
    pyplot.legend(fontsize=14)
    pyplot.xlabel('Epoch', fontsize=16)
    pyplot.ylabel('Model loss', fontsize=16)
    pyplot.xlim([-5, 170])
    pyplot.ylim([0, 0.7])
    pyplot.xticks(fontsize = 14)
    pyplot.yticks(fontsize = 14)
#     pyplot.title(age_group + ': ' + 'model loss', fontsize=16)
    
    pyplot.show()
    
    
    selector.plot(X_rotation=90,which_features='all', X_size = 13, figsize=(5,8))
    